In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.global_attack import DICE

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
data_dr = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading cora dataset...
Selecting 1 largest connected components


In [5]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [6]:
ptb_rate = 0.1

In [7]:
model = DICE()
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(adj, labels, n_perturbations=perturbations)
modified_adj = model.modified_adj

number of pertubations: 506


### arch tests

In [18]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [19]:
model_save_path = "../models/cora_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges.pth"

In [20]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [21]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [22]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [23]:
arr_diff

<2485x2485 sparse matrix of type '<class 'numpy.float32'>'
	with 1012 stored elements in COOrdinate format>

In [24]:
data_modded = copy.deepcopy(data)

In [25]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

10550
1006


In [26]:
data_modded.edge_index.size()

torch.Size([2, 11556])

In [27]:
new_loss, new_accuracy = train.test(data_modded)

print(f"Evasion Change in Accuracy: {(100 * ((new_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Evasion Change in Loss: {(100 * ((new_loss - initial_loss) / initial_loss)):.4f}%")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model_poisoning.reset_parameters()
train_modded = Trainable(model_poisoning)
train_modded.fit(data_modded, 1000)

poisoning_loss, poisoning_accuracy = train_modded.test(data_modded)
print(f"Poisoning Change in Accuracy: {(100 * ((poisoning_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Poisoning Change in Loss: {(100 * ((poisoning_loss - initial_loss) / initial_loss)):.4f}%")

Evasion Change in Accuracy: -0.4963%
Evasion Change in Loss: -2.5823%
Epoch 0, Train Loss - 1.9299582242965698, Val Loss - 1.8596833944320679, Val Accuracy - 0.406
Epoch 20, Train Loss - 0.2518244683742523, Val Loss - 0.8695088624954224, Val Accuracy - 0.736
Epoch 40, Train Loss - 0.0750839114189148, Val Loss - 0.8267417550086975, Val Accuracy - 0.734
Epoch 60, Train Loss - 0.07162811607122421, Val Loss - 0.7942968606948853, Val Accuracy - 0.748
Epoch 80, Train Loss - 0.03167028725147247, Val Loss - 0.7712323665618896, Val Accuracy - 0.754
Epoch 100, Train Loss - 0.03714919090270996, Val Loss - 0.7703956365585327, Val Accuracy - 0.758
Epoch 120, Train Loss - 0.03903079032897949, Val Loss - 0.7812204957008362, Val Accuracy - 0.76
Epoch 140, Train Loss - 0.02727861888706684, Val Loss - 0.789792001247406, Val Accuracy - 0.75
Epoch 160, Train Loss - 0.03804800659418106, Val Loss - 0.7636592388153076, Val Accuracy - 0.762
Epoch 180, Train Loss - 0.04206070676445961, Val Loss - 0.76539504528